# Project Demo

*[Write introduction to the project]*

# How to run the program
1. Install conda
2. conda env create --file environment.yaml
3. Run on command line: `uvicorn main:app --reload --workers 1 --host localhost --port 8008`
4. Run any of the commands below.

## `POST/fit`: Create an ARCH model and fit the data

The following code runs a POST request. This trains an ARCH model and save it in the /models folder.

Parameters:
- `ticker`: a stock ticker
- `n_observations`: the number of past observations to train the model
- `p`,`q`: parameters of the ARCH model

In [3]:
import requests

In [4]:
# Set parameters
ticker="MTNOY"
n_observations=2500
p=3
q=1

# Call fit endpoint
url = "http://localhost:8008/fit"
json = {
    "ticker":ticker,
    "n_observations":n_observations,
    "p":p,
    "q":q
}

response = requests.post(url=url, json=json)
print(response.status_code)

200


In [5]:
response.json()

{'ticker': 'MTNOY',
 'n_observations': 2500,
 'p': 3,
 'q': 1,
 'success': True,
 'message': "Trained and saved 'models\\2025-09-09T15-40-58.148031-ARCH31_MTNOY.pkl'. Metrics: AIC 9895.361316315586, BIC 9928.966731072838."}

# POST/predict 

Gets predictions from the model.

Parameters:
- ticker: a stock ticker
- n_days: number of days to predict
- use_model: which model to use ("latest" for the latest model in the /models folder)

In [6]:
# Set parameters
ticker="MTNOY"
n_days=20
use_model = "latest"

# Call predict API
url = "http://localhost:8008/predict"
json = {
    "ticker":ticker,
    "n_days": n_days,
    "use_model": use_model
}

response = requests.post(url=url, json=json)
print(response.status_code)

200


In [7]:
response.json()

{'ticker': 'MTNOY',
 'n_days': 20,
 'use_model': 'latest',
 'success': True,
 'forecast': {'2023-09-11T00:00:00': 2.420952310004812,
  '2023-09-12T00:00:00': 2.44342149905873,
  '2023-09-13T00:00:00': 2.4649664904541893,
  '2023-09-14T00:00:00': 2.4856344764141203,
  '2023-09-15T00:00:00': 2.505469391354207,
  '2023-09-18T00:00:00': 2.5245122184063375,
  '2023-09-19T00:00:00': 2.5428012587853823,
  '2023-09-20T00:00:00': 2.5603723695204135,
  '2023-09-21T00:00:00': 2.5772591740380433,
  '2023-09-22T00:00:00': 2.593493249358153,
  '2023-09-25T00:00:00': 2.6091042930568555,
  '2023-09-26T00:00:00': 2.62412027265632,
  '2023-09-27T00:00:00': 2.638567559693833,
  '2023-09-28T00:00:00': 2.652471050385853,
  '2023-09-29T00:00:00': 2.665854274523209,
  '2023-10-02T00:00:00': 2.6787394940002707,
  '2023-10-03T00:00:00': 2.6911477921853693,
  '2023-10-04T00:00:00': 2.7030991551751513,
  '2023-10-05T00:00:00': 2.7146125458364683,
  '2023-10-06T00:00:00': 2.725705971421407},
 'message': ''}